In [1]:
import numpy as np
import pandas as pd
import nltk
import gensim
import pyemd
import sklearn
import re
from gensim import corpora, models, similarities
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.corpus import stopwords
from sklearn.decomposition import TruncatedSVD
import pickle
import networkx as nx
from networkx.algorithms.shortest_paths.generic import shortest_path

D:\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
features = pd.read_pickle('features.pkl')
stop = set(stopwords.words('english')+['1','2','3','4','5','6','7','8','9','0'])

In [3]:
clean_title = pd.read_pickle('clean_title.pkl')['text'].values.tolist()
clean_abstract = pd.read_pickle('clean_abstract.pkl')['text'].values.tolist()

# Data import

In [4]:
columes = ["id", "year", "title", "authors", "journal", "abstract"]
df= pd.read_csv("dataset/node_information.csv",names=columes)

In [5]:
df['title'] = clean_title
df['abstract'] = clean_abstract

In [6]:
df.to_pickle('df_clean.pkl')

In [7]:
data_set = np.loadtxt("dataset/training_set.txt").astype(np.int64)
data = data_set[:, :2]
test = np.loadtxt("dataset/testing_set.txt").astype(np.int64)
label = data_set[:, -1]

In [8]:
data_dic = {}

for index, row in df.iterrows():
    data_dic[row['id']] = (row['year'], row['title'], row['authors'], row['journal'], row['abstract'])

In [41]:
features = pd.DataFrame()
features['year1'] = [data_dic[id][0] for id in data[:, 0]]
features['year2'] = [data_dic[id][0] for id in data[:, 1]]

In [43]:
features['year_differ'] = list(map(lambda x: abs(x[0]-x[1]), zip(features['year1'].values.tolist(), features['year2'].values.tolist())))

In [16]:
features['year_max'] = list(map(lambda x: max(x[0], x[1]), zip(features['year1'].values.tolist(), features['year2'].values.tolist())))
features['year_min'] = list(map(lambda x: min(x[0], x[1]), zip(features['year1'].values.tolist(), features['year2'].values.tolist())))

In [53]:
def sameAuthors(author1, author2):
    removed1 = re.sub(r" ?\([^)]+\)", "", author1)
    removed2 = re.sub(r" ?\([^)]+\)", "", author2)
    splited1 = removed1.split(', ')
    splited2 = removed2.split(', ')
    count = 0
    for a1 in splited1:
        for a2 in splited2:
            if a1 == a2: count += 1
    return count

In [54]:
temp = []
for id1, id2 in data:
    if isinstance(data_dic[id1][2], float) or isinstance(data_dic[id2][2], float):
        temp.append(-1)
    else:
        temp.append(sameAuthors(data_dic[id1][2], data_dic[id2][2]))
features['sameAutCount'] = temp

** authors **

In [99]:
id_aut_dic = {}
id_aut_list = []
for index, row in df.iterrows():
    all_auts = row['authors']
    if not isinstance(all_auts, float):
        splited_auts = re.sub(r" ?\([^)]*\)", "", all_auts.strip()).split(',')
        temp = []
        for aut in splited_auts:
            aut_re1 = re.sub(r" ?\([^)]*", "", aut)
            aut_re2 = re.sub(r" ?", "", aut_re1)
            if aut_re2 != '':
                temp.append(aut_re2)
        id_aut_dic[row['id']] = temp
        id_aut_list.append(temp)
    else:
        id_aut_dic[row['id']]=[]
        id_aut_list.append([])

In [103]:
df['authors_list']=id_aut_list

In [104]:
len(id_aut_dic)

27770

In [105]:
aut_id_dic = {}
aut_set = set([])
for index, row in df.iterrows():
    for aut in row['authors_list']:
        aut_set.add(aut)
        if aut in aut_id_dic:
            aut_id_dic[aut].append(row['id'])
        else:
            aut_id_dic[aut] = [row['id']]

In [106]:
temp = []
for id1, id2 in data:
    c = 0
    for aut in id_aut_dic[id1]:
        if aut in id_aut_dic[id2]: c += 1
    temp.append(c)

In [107]:
features['sameAut']=temp

In [177]:
features.to_pickle('features.pkl')

In [109]:
aut_G = nx.Graph()
for aut in aut_set:
    aut_G.add_node(aut)
count = 0
for id1, id2, lab in data_set:
    if lab==1:
        for aut1 in id_aut_dic[id1]:
            for aut2 in id_aut_dic[id2]:
                if aut_G.has_edge(aut1, aut2):
                    aut_G[aut1][aut2]['w'] += 1
                else:
                    aut_G.add_edge(aut1, aut2, {'w': 1})
    count+=1
    if count%2000==0: print(count/6155.12)

In [125]:
aut_G['JohnJ.Oh']

{'A.A.Deriglazov': {'w': 1},
 'A.Chamblin': {'w': 2},
 'A.Fabbri': {'w': 4},
 'A.J.M.Medved': {'w': 2},
 'A.Shirzad': {'w': 1},
 'AkiraKokado': {'w': 2},
 'AlainConnes': {'w': 1},
 'AlbertSchwarz': {'w': 1},
 'AndreiO.Starinets': {'w': 1},
 'AndrewChamblin': {'w': 1},
 'AndrewStrominger': {'w': 5},
 'BertJanssen': {'w': 1},
 'BinWang': {'w': 1},
 'BiswajitChakraborty': {'w': 1},
 'C.Neves': {'w': 1},
 'CarlosHerdeiro': {'w': 1},
 'CedricDeffayet': {'w': 3},
 'CesarGomez': {'w': 1},
 'Chong-SunChu': {'w': 2},
 'ChrisHull': {'w': 1},
 'ClaudioTeitelboim': {'w': 5},
 'CsabaCsaki': {'w': 1},
 'D.G.Pak': {'w': 1},
 'D.Grumiller': {'w': 1},
 'D.J.Navarro': {'w': 3},
 'D.V.Vassilevich': {'w': 1},
 'DanielaBigatti': {'w': 2},
 'DannyBirmingham': {'w': 5},
 'DavidLanglois': {'w': 2},
 'DeanL.Welch': {'w': 2},
 'DmitriFursaev': {'w': 1},
 'E.Katz': {'w': 1},
 'EdwardWitten': {'w': 1},
 'ElcioAbdalla': {'w': 2},
 'EliasC.Vagenas': {'w': 1},
 'F.Ardalan': {'w': 1},
 'FinnLarsen': {'w': 1},
 'GWGib

In [145]:
aut_refer_list_dic = {}
aut_refer_set_dic = {}
for aut in aut_set:
    aut_refer_list_dic[aut] = aut_G.neighbors(aut)
    aut_refer_set_dic[aut] = set(aut_G.neighbors(aut))

In [141]:
len(aut_refer_dic['PaulS.Aspinwall'])

453

In [144]:
len(dic_edge_list[1001])

90

** D==A--A--A==D **

In [151]:
temp = []
count = 0
for id1, id2 in data:
    c = 0
    for aut1 in id_aut_dic[id1]:
        for aut2 in id_aut_dic[id2]:
            for aut3 in aut_refer_list_dic[aut1]:
                if aut3 != aut2 and aut3!=aut1 and aut2 in aut_refer_set_dic[aut3]:
                    c += 1
    temp.append(c)
    count+=1
    if count%200==0: print(count/6155.12)

0.3249327389230429
0.6498654778460858
0.9747982167691288
1.2997309556921717
1.6246636946152146
1.9495964335382576
2.2745291724613006
2.5994619113843433
2.9243946503073865
3.2493273892304293
3.5742601281534725
3.8991928670765152
4.224125605999558
4.549058344922601
4.873991083845644
5.198923822768687
5.52385656169173
5.848789300614773
6.173722039537815
6.498654778460859
6.823587517383902
7.148520256306945
7.473452995229987
7.7983857341530305
8.123318473076074
8.448251211999116
8.77318395092216
9.098116689845202
9.423049428768245
9.747982167691289
10.072914906614331
10.397847645537373
10.722780384460417
11.04771312338346
11.372645862306502
11.697578601229546
12.022511340152588
12.34744407907563
12.672376817998675
12.997309556921717
13.32224229584476
13.647175034767804
13.972107773690846
14.29704051261389
14.621973251536932
14.946905990459975
15.271838729383019
15.596771468306061
15.921704207229103
16.246636946152147
16.57156968507519
16.896502423998232
17.221435162921274
17.54636790184432

In [152]:
len(temp)

615512

In [153]:
features['AAA_edge'] = temp

** D==A--D--A==D **

In [157]:
AD_list_dic = {aut: [] for aut in aut_set}
AD_set_dic = {aut: set([]) for aut in aut_set}
count = 0
for id1, id2, lab in data_set:
    if lab==1:
        for aut1 in id_aut_dic[id1]:
            if id2 not in AD_set_dic[aut1]:
                AD_list_dic[aut1].append(id2)
                AD_set_dic[aut1].add(id2)
        for aut2 in id_aut_dic[id2]:
            if id1 not in AD_set_dic[aut2]:
                AD_list_dic[aut2].append(id1)
                AD_set_dic[aut2].add(id1)
    count+=1
    if count%2000==0: print(count/6155.12)

0.3249327389230429
0.6498654778460858
0.9747982167691288
1.2997309556921717
1.6246636946152146
1.9495964335382576
2.2745291724613006
2.5994619113843433
2.9243946503073865
3.2493273892304293
3.5742601281534725
3.8991928670765152
4.224125605999558
4.549058344922601
4.873991083845644
5.198923822768687
5.52385656169173
5.848789300614773
6.173722039537815
6.498654778460859
6.823587517383902
7.148520256306945
7.473452995229987
7.7983857341530305
8.123318473076074
8.448251211999116
8.77318395092216
9.098116689845202
9.423049428768245
9.747982167691289
10.072914906614331
10.397847645537373
10.722780384460417
11.04771312338346
11.372645862306502
11.697578601229546
12.022511340152588
12.34744407907563
12.672376817998675
12.997309556921717
13.32224229584476
13.647175034767804
13.972107773690846
14.29704051261389
14.621973251536932
14.946905990459975
15.271838729383019
15.596771468306061
15.921704207229103
16.246636946152147
16.57156968507519
16.896502423998232
17.221435162921274
17.54636790184432

In [170]:
temp = []
count = 0
for id1, id2 in data:
    c = 0
    for aut1 in id_aut_dic[id1]:
        for aut2 in id_aut_dic[id2]:
            for id3 in AD_list_dic[aut1]:
                if id3 != id1 and id3 != id2 and id3 in AD_set_dic[aut2]:
                    c += 1
    temp.append(c)
    count+=1
    if count%2000==0: print(count/6155.12)

0.3249327389230429
0.6498654778460858
0.9747982167691288
1.2997309556921717
1.6246636946152146
1.9495964335382576
2.2745291724613006
2.5994619113843433
2.9243946503073865
3.2493273892304293
3.5742601281534725
3.8991928670765152
4.224125605999558
4.549058344922601
4.873991083845644
5.198923822768687
5.52385656169173
5.848789300614773
6.173722039537815
6.498654778460859
6.823587517383902
7.148520256306945
7.473452995229987
7.7983857341530305
8.123318473076074
8.448251211999116
8.77318395092216
9.098116689845202
9.423049428768245
9.747982167691289
10.072914906614331
10.397847645537373
10.722780384460417
11.04771312338346
11.372645862306502
11.697578601229546
12.022511340152588
12.34744407907563
12.672376817998675
12.997309556921717
13.32224229584476
13.647175034767804
13.972107773690846
14.29704051261389
14.621973251536932
14.946905990459975
15.271838729383019
15.596771468306061
15.921704207229103
16.246636946152147
16.57156968507519
16.896502423998232
17.221435162921274
17.54636790184432

In [171]:
features['ADA_edge'] = temp

** D--A--D **

In [ ]:
DA_list_dic = {id: [] for id in df['id'].values.tolist()}
count = 0
for id1, id2, lab in data_set:
    if lab==1:
        for aut1 in id_aut_dic[id1]:
            if id2 not in AD_set_dic[aut1]:
                AD_list_dic[aut1].append(id2)
                AD_set_dic[aut1].add(id2)
        for aut2 in id_aut_dic[id2]:
            if id1 not in AD_set_dic[aut2]:
                AD_list_dic[aut2].append(id1)
                AD_set_dic[aut2].add(id1)
    count+=1
    if count%2000==0: print(count/6155.12)

In [173]:
temp = []
count = 0
for id1, id2 in data:
    c = 0
    for aut in aut_set:
        if aut not in id_aut_dic[id1] and aut not in id_aut_dic[id2]:
            if id1 in AD_set_dic[aut] and id2 in AD_set_dic[aut]:
                c += 1
    temp.append(c)
    count+=1
    if count%2000==0: print(count/6155.12)

KeyboardInterrupt: 

** D==A==D==A==D **

In [174]:
temp = []
count = 0
for id1, id2 in data:
    c = 0
    for aut1 in id_aut_dic[id1]:
        for aut2 in id_aut_dic[id2]:
            for id3 in aut_id_dic[aut1]:
                if id3 in aut_id_dic[aut2]:
                    c += 1
    temp.append(c)
    count+=1
    if count%2000==0: print(count/6155.12)

0.3249327389230429
0.6498654778460858
0.9747982167691288
1.2997309556921717
1.6246636946152146
1.9495964335382576
2.2745291724613006
2.5994619113843433
2.9243946503073865
3.2493273892304293
3.5742601281534725
3.8991928670765152
4.224125605999558
4.549058344922601
4.873991083845644
5.198923822768687
5.52385656169173
5.848789300614773
6.173722039537815
6.498654778460859
6.823587517383902
7.148520256306945
7.473452995229987
7.7983857341530305
8.123318473076074
8.448251211999116
8.77318395092216
9.098116689845202
9.423049428768245
9.747982167691289
10.072914906614331
10.397847645537373
10.722780384460417
11.04771312338346
11.372645862306502
11.697578601229546
12.022511340152588
12.34744407907563
12.672376817998675
12.997309556921717
13.32224229584476
13.647175034767804
13.972107773690846
14.29704051261389
14.621973251536932
14.946905990459975
15.271838729383019
15.596771468306061
15.921704207229103
16.246636946152147
16.57156968507519
16.896502423998232
17.221435162921274
17.54636790184432

In [176]:
features['=A=D=A='] = temp

In [181]:
features[['=A=D=A=','sameAut']]

,=A=D=A=,sameAut
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
5,0,0
6,0,0
7,0,0
8,0,0
9,0,0


In [24]:
a = re.sub(r" ?\([^)]+\)", "", 'R. Caseiro (Coimbra), J.-P. Francoise (Paris VI), R. Sasaki (Kyoto)').split(',')
for aut in a:
        aut = aut.strip()
print(a)

['R. Caseiro', ' J.-P. Francoise', ' R. Sasaki']


In [55]:
dic_aut = {}
for index, row in df.iterrows():
    names = row['authors_list']
    if not isinstance(names, float):
        splited_names = re.sub(r" ?\([^)]+\)", "", names).split(', ')
        for name in splited_names:
            if name not in dic_aut:
                dic_aut[name] = 1
            else:
                dic_aut[name] += 1


In [62]:
temp_a1 = []
temp_a2 = []
for id1, id2 in data:
    a1 = data_dic[id1][2]
    a2 = data_dic[id2][2]
    if isinstance(a1, float):
        temp_a1.append(-1)
    else:
        splited_a1 = re.sub(r" ?\([^)]+\)", "", a1).split(', ')
        sum_a1 = 0
        for a in splited_a1:
            sum_a1 += dic_aut[a]
        temp_a1.append(sum_a1)
    if isinstance(a2, float):
        temp_a2.append(-1)
    else:
        splited_a2 = re.sub(r" ?\([^)]+\)", "", a2).split(', ')
        sum_a2 = 0
        for a in splited_a2:
            sum_a2 += dic_aut[a]
        temp_a2.append(sum_a2)
features['aut1_sum'] = temp_a1
features['aut2_sum'] = temp_a2

In [66]:
features['aut_sum'] = list(map(lambda x: abs(x[0]-x[1]), zip(features['aut1_sum'].values.tolist(), features['aut2_sum'].values.tolist())))

In [57]:
dic_doc = {id: 0 for id in df['id'].values.tolist()}
for id1, id2, lab in data_set[:, :]:
    if lab == 1:
        dic_doc[id1] += 1
        dic_doc[id2] += 1

In [42]:
st_count = {id: [0,0] for id in df['id'].values.tolist()}
for id1, id2, lab in data_set[:, :]:
    if lab == 1:
        st_count[id1][0] += 1
        st_count[id2][1] += 1

In [43]:
st_count

{1001: [80, 10],
 1002: [35, 42],
 1003: [1, 14],
 1004: [1, 1],
 1005: [20, 1],
 1006: [1, 1],
 1007: [4, 0],
 1008: [6, 2],
 1009: [1, 1],
 1010: [5, 12],
 1011: [19, 4],
 1012: [10, 37],
 1013: [11, 16],
 1014: [8, 3],
 1015: [0, 0],
 1016: [18, 8],
 1017: [8, 5],
 1018: [18, 10],
 1019: [7, 6],
 1020: [1, 0],
 1021: [15, 2],
 1022: [3, 1],
 1023: [6, 8],
 1024: [18, 25],
 1025: [38, 17],
 1026: [6, 0],
 1027: [17, 54],
 1028: [6, 1],
 1029: [32, 7],
 1030: [3, 0],
 1031: [3, 4],
 1032: [6, 33],
 1033: [60, 107],
 1034: [4, 0],
 1035: [7, 50],
 1036: [16, 11],
 1037: [22, 29],
 1038: [2, 1],
 1039: [2, 4],
 1040: [1, 1],
 1041: [46, 29],
 1042: [12, 2],
 1044: [14, 3],
 1045: [3, 1],
 1046: [6, 6],
 1047: [9, 3],
 1048: [4, 2],
 1049: [28, 11],
 1050: [2, 6],
 1051: [22, 2],
 1052: [2, 0],
 1053: [18, 103],
 1054: [42, 8],
 1055: [29, 25],
 1056: [12, 21],
 1057: [27, 2],
 1060: [5, 2],
 1061: [2, 1],
 1062: [10, 7],
 1063: [15, 15],
 1064: [13, 2],
 1065: [17, 4],
 1066: [34, 12],


In [58]:
temp1 = []
temp2 = []
for id1, id2, lab in data_set:
    temp1.append(dic_doc[id1]-lab)
    temp2.append(dic_doc[id2]-lab)
features['connection1'] = temp1
features['connection2'] = temp2

In [27]:
features = features.drop(['edge2_dir'], axis = 1)

In [61]:
features.to_pickle('features.pkl')

In [14]:
features['connex_max'] = list(map(lambda x: max(x[0], x[1]), zip(features['connection1'].values.tolist(), features['connection2'].values.tolist())))
features['connex_min'] = list(map(lambda x: min(x[0], x[1]), zip(features['connection1'].values.tolist(), features['connection2'].values.tolist())))

In [60]:
temp1 = []
temp2 = []
temp3 = []
temp4 = []
for id1, id2, lab in data_set:
    temp1.append(st_count[id1][0]-lab)
    temp2.append(st_count[id2][0])
    temp3.append(st_count[id1][1])
    temp4.append(st_count[id2][1]-lab)
features['src1'] = temp1
features['tgt2'] = temp4

In [82]:
dic_doclen = {df['id'].values.tolist()[i]: len(tok_corp[i]) for i in range(df.shape[0])}

In [84]:
temp1 = []
temp2 = []
for id1, id2 in data[:, :]:
    temp1.append(dic_doclen[id1])
    temp2.append(dic_doclen[id2])
features['len1'] = temp1
features['len2'] = temp2

In [143]:
dic_edge_list = {id: [] for id in df['id'].values.tolist()}
dic_edge_set = {id: set([]) for id in df['id'].values.tolist()}
for id1, id2, lab in data_set:
    if lab == 1:
        dic_edge_list[id1].append(id2)
        dic_edge_set[id1].add(id2)
        dic_edge_list[id2].append(id1)
        dic_edge_set[id2].add(id1)

** source-target **

In [35]:
src_tgt_list = {id: [] for id in df['id'].values.tolist()}
src_tgt_set = {id: set([]) for id in df['id'].values.tolist()}
tgt_src_list = {id: [] for id in df['id'].values.tolist()}
for id1, id2, lab in data_set:
    if lab == 1:
        src_tgt_list[id1].append(id2)
        src_tgt_set[id1].add(id2)
        tgt_src_list[id2].append(id1)

** edge2 **

In [65]:
temp = []
for id1, id2 in data:
    c = 0
    for id3 in dic_edge_list[id1]:
        if id3 != id2 and id2 in dic_edge_set[id3]:
            c += 1
    temp.append(c)

In [21]:
features['edge2'] = temp

NameError: name 'temp' is not defined

** edge2_dir **

In [29]:
temp = []
for id1, id2 in data:
    c = 0
    for id3 in src_tgt_list[id1]:
        if id3 != id2 and id2 in src_tgt_set[id3]:
            c += 1
    temp.append(c)

In [30]:
features['->D->'] = temp

** ->D<- **

In [31]:
temp = []
for id1, id2 in data:
    c = 0
    for id3 in src_tgt_list[id1]:
        if id3 != id2 and id3 in src_tgt_set[id2]:
            c += 1
    temp.append(c)

In [32]:
features['->D<-'] = temp

In [37]:
temp = []
for id1, id2 in data:
    c = 0
    for id3 in tgt_src_list[id1]:
        if id3 != id2 and id2 in src_tgt_set[id3]:
            c += 1
    temp.append(c)

In [38]:
features['<-D->'] = temp

In [136]:
temp = []
for id1, id2 in data:
    c = 0
    for id3 in dic_edge_list[id1]:
        if id3 != id2 and id2 in dic_edge_set[id3]:
            c += G.degree(id3)
    temp.append(c)

NameError: name 'dic_edge_list' is not defined

In [78]:
features['edge2_weighted'] = temp

In [47]:
temp = []
for id1, id2 in data:
    c = 0
    for id3 in src_tgt_list[id1]:
        if id3 != id2 and id2 in src_tgt_set[id3]:
            c += st_count[id3][0]*st_count[id3][1]
    temp.append(c)

In [48]:
features['->D->_w'] = temp

In [208]:
temp = []
for id1, id2 in data:
    c = 0
    for id3 in dic_edge_list[id1]:
        if id3 != id2 and id2 in dic_edge_set[id3]:
            c += dic_doc[id1]*dic_doc[id2]*dic_doc[id3]
    temp.append(c)

In [209]:
features['edge2_123_weighted'] = temp

In [210]:
temp = []
for id1, id2 in data:
    c = 0
    for id3 in dic_edge_list[id1]:
        if id3 != id2 and id2 in dic_edge_set[id3]:
            c += dic_doc[id3]*dic_doc[id3]
    temp.append(c)

In [211]:
features['edge2_sqr_weighted'] = temp

In [188]:
temp = []
count = 0
for id1, id2 in data:
    c = 0
    for id3 in dic_edge_list[id1]:
        if id3 != id2:
            for id4 in dic_edge_list[id3]:
                if id4 != id2 and id4 != id1 and id2 in dic_edge_set[id4]:
                    c += 1
    temp.append(c)
    count += 1
    if count%1000 == 0:
        print(count/6155.12)

0.16246636946152146


KeyboardInterrupt: 

In [ ]:
features['edge3'] = temp

In [212]:
features.to_pickle('features.pkl')

In [52]:
temp = []
count = 0
for id1, id2 in data:
    c = 0
    for id3 in src_tgt_list[id1]:
        if id3 != id2:
            for id4 in src_tgt_list[id3]:
                if id4 != id2 and id4 != id1 and id2 in src_tgt_set[id4]:
                    c += 1
    temp.append(c)
    count += 1
    if count%1000 == 0:
        print("%.3f%%" % count/6155.12)

0.16246636946152146
0.3249327389230429
0.4873991083845644
0.6498654778460858
0.8123318473076073
0.9747982167691288
1.1372645862306503
1.2997309556921717
1.4621973251536933
1.6246636946152146
1.7871300640767362
1.9495964335382576
2.112062802999779
2.2745291724613006
2.436995541922822
2.5994619113843433
2.761928280845865
2.9243946503073865
3.0868610197689077
3.2493273892304293
3.411793758691951
3.5742601281534725
3.7367264976149936
3.8991928670765152
4.061659236538037
4.224125605999558
4.38659197546108
4.549058344922601
4.711524714384122
4.873991083845644
5.0364574533071655
5.198923822768687
5.361390192230209
5.52385656169173
5.686322931153251
5.848789300614773
6.011255670076294
6.173722039537815
6.336188408999337
6.498654778460859
6.66112114792238
6.823587517383902
6.986053886845423
7.148520256306945
7.310986625768466
7.473452995229987
7.635919364691509
7.7983857341530305
7.960852103614552
8.123318473076074
8.285784842537595
8.448251211999116
8.610717581460637
8.77318395092216
8.9356503

72.78493351876162
72.94739988822315
73.10986625768466
73.27233262714618
73.4347989966077
73.59726536606922
73.75973173553075
73.92219810499226
74.08466447445379
74.24713084391531
74.40959721337683
74.57206358283835
74.73452995229988
74.89699632176139
75.05946269122292
75.22192906068445
75.38439543014596
75.54686179960748
75.70932816906901
75.87179453853052
76.03426090799205
76.19672727745356
76.35919364691509
76.52166001637661
76.68412638583813
76.84659275529965
77.00905912476118
77.17152549422269
77.33399186368422
77.49645823314574
77.65892460260726
77.82139097206878
77.98385734153031
78.14632371099182
78.30879008045335
78.47125644991488
78.63372281937639
78.79618918883791
78.95865555829943
79.12112192776095
79.28358829722248
79.44605466668399
79.60852103614552
79.77098740560704
79.93345377506856
80.09592014453008
80.25838651399161
80.42085288345312
80.58331925291465
80.74578562237618
80.90825199183769
81.07071836129921
81.23318473076074
81.39565110022225
81.55811746968378
81.72058383

In [53]:
features['->D->D->'] = temp
features.to_pickle('features.pkl')

In [ ]:
temp = []
count = 0
for id1, id2 in data:
    c = 0
    for id3 in src_tgt_list[id1]:
        if id3 != id2:
            for id4 in src_tgt_list[id3]:
                if id4 != id2 and id4 != id1:
                    for id5 in src_tgt_list[id4]:
                        if id5 != id2 and id5 != id1 and id5!=id3 and id2 in src_tgt_set[id5]:
                            c +=1
    temp.append(c)
    count += 1
    if count%1000 == 0:
        print(count/6155.12)

0.16246636946152146
0.3249327389230429
0.4873991083845644
0.6498654778460858
0.8123318473076073
0.9747982167691288
1.1372645862306503
1.2997309556921717
1.4621973251536933
1.6246636946152146
1.7871300640767362
1.9495964335382576
2.112062802999779
2.2745291724613006
2.436995541922822
2.5994619113843433
2.761928280845865
2.9243946503073865
3.0868610197689077
3.2493273892304293
3.411793758691951
3.5742601281534725
3.7367264976149936
3.8991928670765152
4.061659236538037
4.224125605999558
4.38659197546108
4.549058344922601
4.711524714384122
4.873991083845644
5.0364574533071655
5.198923822768687
5.361390192230209
5.52385656169173
5.686322931153251
5.848789300614773
6.011255670076294
6.173722039537815
6.336188408999337
6.498654778460859
6.66112114792238
6.823587517383902
6.986053886845423
7.148520256306945
7.310986625768466
7.473452995229987
7.635919364691509
7.7983857341530305
7.960852103614552
8.123318473076074
8.285784842537595
8.448251211999116
8.610717581460637


In [ ]:
features['->D->D->D->'] = temp
features.to_pickle('features.pkl')

In [192]:
temp = []
count = 0
for id1, id2 in data:
    c = 0
    for id3 in dic_edge_list[id1]:
        if id3 != id2:
            for id4 in dic_edge_list[id3]:
                if id4 != id2 and id4 != id1 and id2 in dic_edge_set[id4]:
                    c += dic_doc[id3]*dic_doc[id4]
    temp.append(c)
    count += 1
    if count%1000 == 0:
        print(count/6155.12)

0.16246636946152146
0.3249327389230429
0.4873991083845644
0.6498654778460858
0.8123318473076073
0.9747982167691288
1.1372645862306503
1.2997309556921717
1.4621973251536933
1.6246636946152146
1.7871300640767362
1.9495964335382576
2.112062802999779
2.2745291724613006
2.436995541922822
2.5994619113843433
2.761928280845865
2.9243946503073865
3.0868610197689077
3.2493273892304293
3.411793758691951
3.5742601281534725
3.7367264976149936
3.8991928670765152
4.061659236538037
4.224125605999558
4.38659197546108
4.549058344922601
4.711524714384122
4.873991083845644
5.0364574533071655
5.198923822768687
5.361390192230209
5.52385656169173
5.686322931153251
5.848789300614773
6.011255670076294
6.173722039537815
6.336188408999337
6.498654778460859
6.66112114792238
6.823587517383902
6.986053886845423
7.148520256306945
7.310986625768466
7.473452995229987
7.635919364691509
7.7983857341530305
7.960852103614552
8.123318473076074
8.285784842537595
8.448251211999116
8.610717581460637
8.77318395092216
8.9356503

72.78493351876162
72.94739988822315
73.10986625768466
73.27233262714618
73.4347989966077
73.59726536606922
73.75973173553075
73.92219810499226
74.08466447445379
74.24713084391531
74.40959721337683
74.57206358283835
74.73452995229988
74.89699632176139
75.05946269122292
75.22192906068445
75.38439543014596
75.54686179960748
75.70932816906901
75.87179453853052
76.03426090799205
76.19672727745356
76.35919364691509
76.52166001637661
76.68412638583813
76.84659275529965
77.00905912476118
77.17152549422269
77.33399186368422
77.49645823314574
77.65892460260726
77.82139097206878
77.98385734153031
78.14632371099182
78.30879008045335
78.47125644991488
78.63372281937639
78.79618918883791
78.95865555829943
79.12112192776095
79.28358829722248
79.44605466668399
79.60852103614552
79.77098740560704
79.93345377506856
80.09592014453008
80.25838651399161
80.42085288345312
80.58331925291465
80.74578562237618
80.90825199183769
81.07071836129921
81.23318473076074
81.39565110022225
81.55811746968378
81.72058383

In [ ]:
features['edge3_weighted'] = temp

In [51]:
features.to_pickle('features.pkl')

In [55]:
features[['edge3','->D->D->']]

,edge3,->D->D->
0,0,0
1,1417,43
2,0,0
3,24,0
4,1,0
5,328,0
6,1483,195
7,0,0
8,433,4
9,0,0


In [150]:
dic_edge_set_2 = {id: set([]) for id in df['id'].values.tolist()}
count=0
for id1 in df['id'].values.tolist():
    for id2 in dic_edge_list[id1]:
        dic_edge_set_2[id1].add(id2)
        for id3 in dic_edge_list[id2]:
            if id3 != id1:
                dic_edge_set_2[id1].add(id3)
    count+=1
    if count%500==0: print(count/277.7)

1.8005041411595248
3.6010082823190497
5.401512423478574
7.202016564638099
9.002520705797624
10.803024846957149
12.603528988116674
14.404033129276199
16.204537270435722
18.005041411595247
19.805545552754772
21.606049693914297
23.406553835073822
25.207057976233347
27.007562117392872
28.808066258552397
30.60857039971192
32.409074540871444
34.20957868203097
36.010082823190494
37.81058696435002
39.611091105509544
41.411595246669066
43.212099387828594
45.012603528988116
46.813107670147645
48.613611811307166
50.414115952466695
52.214620093626216
54.015124234785745
55.815628375945266
57.616132517104795
59.416636658264316
61.21714079942384
63.01764494058337
64.81814908174289
66.61865322290241
68.41915736406195
70.21966150522147
72.02016564638099
73.82066978754051
75.62117392870005
77.42167806985957
79.22218221101909
81.02268635217861
82.82319049333813
84.62369463449767
86.42419877565719
88.22470291681671
90.02520705797623
91.82571119913577
93.62621534029529
95.42671948145481
97.22722362261433
9

In [163]:
dic_edge_list_2 = {id: [] for id in df['id'].values.tolist()}
count=0
for id1 in df['id'].values.tolist():
    for id2 in dic_edge_list[id1]:
        dic_edge_list_2[id1].append(id2)
        for id3 in dic_edge_list[id2]:
            if id3 != id1:
                dic_edge_list_2[id1].append(id3)
    count+=1
    if count%500==0: print(count/277.7)

1.8005041411595248
3.6010082823190497
5.401512423478574
7.202016564638099
9.002520705797624
10.803024846957149
12.603528988116674
14.404033129276199
16.204537270435722
18.005041411595247
19.805545552754772
21.606049693914297
23.406553835073822
25.207057976233347
27.007562117392872
28.808066258552397
30.60857039971192
32.409074540871444
34.20957868203097
36.010082823190494
37.81058696435002
39.611091105509544
41.411595246669066
43.212099387828594
45.012603528988116
46.813107670147645
48.613611811307166
50.414115952466695
52.214620093626216
54.015124234785745
55.815628375945266
57.616132517104795
59.416636658264316
61.21714079942384
63.01764494058337
64.81814908174289
66.61865322290241
68.41915736406195
70.21966150522147
72.02016564638099
73.82066978754051
75.62117392870005
77.42167806985957
79.22218221101909
81.02268635217861
82.82319049333813
84.62369463449767
86.42419877565719
88.22470291681671
90.02520705797623
91.82571119913577
93.62621534029529
95.42671948145481
97.22722362261433
9

In [165]:
dic_edge_set_3 = {id: set([]) for id in df['id'].values.tolist()}
id_list = df['id'].values.tolist()
count=0
for id1 in id_list:
    for id2 in dic_edge_set[id1]:
        dic_edge_set_3[id1].add(id2)
        for id3 in dic_edge_set_2[id2]:
            if id3 != id1:
                dic_edge_set_3[id1].add(id3)
    count+=1
    if count%100==0: print(count/277.7)

0.3601008282319049
0.7202016564638098
1.0803024846957148
1.4404033129276197
1.8005041411595248
2.1606049693914295
2.5207057976233345
2.8808066258552394
3.2409074540871443
3.6010082823190497
3.9611091105509546
4.321209938782859
4.6813107670147645
5.041411595246669
5.401512423478574
5.761613251710479
6.121714079942384
6.481814908174289
6.841915736406194
7.202016564638099
7.562117392870004
7.922218221101909
8.282319049333815
8.642419877565718
9.002520705797624
9.362621534029529
9.722722362261434
10.082823190493338
10.442924018725243
10.803024846957149
11.163125675189054
11.523226503420958
11.883327331652863
12.243428159884768
12.603528988116674
12.963629816348577
13.323730644580483
13.683831472812388
14.043932301044293
14.404033129276199
14.764133957508102
15.124234785740008
15.484335613971913
15.844436442203818
16.204537270435722
16.56463809866763
16.924738926899533
17.284839755131436
17.644940583363343
18.005041411595247
18.36514223982715
18.725243068059058
19.08534389629096
19.44544472

In [156]:
temp = []
id_list = df['id'].values.tolist()
count=0
for id1, id2 in data:
    c = 0
    for id3 in id_list:
        if id3 in dic_edge_set_2[id1] and id3 in dic_edge_set_2[id2]:
            c += 1
    temp.append(c)
    count+=1
    if count%2000==0: print(count/6155.12)

0.3249327389230429
0.6498654778460858
0.9747982167691288
1.2997309556921717
1.6246636946152146
1.9495964335382576
2.2745291724613006
2.5994619113843433
2.9243946503073865
3.2493273892304293
3.5742601281534725
3.8991928670765152
4.224125605999558
4.549058344922601
4.873991083845644
5.198923822768687
5.52385656169173
5.848789300614773
6.173722039537815
6.498654778460859
6.823587517383902
7.148520256306945
7.473452995229987
7.7983857341530305
8.123318473076074
8.448251211999116
8.77318395092216
9.098116689845202
9.423049428768245
9.747982167691289
10.072914906614331
10.397847645537373
10.722780384460417
11.04771312338346
11.372645862306502
11.697578601229546
12.022511340152588
12.34744407907563
12.672376817998675
12.997309556921717
13.32224229584476
13.647175034767804
13.972107773690846
14.29704051261389
14.621973251536932
14.946905990459975
15.271838729383019
15.596771468306061
15.921704207229103
16.246636946152147
16.57156968507519
16.896502423998232
17.221435162921274
17.54636790184432

In [157]:
edge4 = pd.DataFrame()
edge4['edge4'] = temp
edge4.to_pickle('edge4.pkl')

In [159]:
features['edge4'] = edge4['edge4']

In [ ]:
temp = []
id_list = df['id'].values.tolist()
count=0
for id1, id2 in data:
    c = 0
    for id3 in id_list:
        if id3 in dic_edge_set_1[id1] and id3 in dic_edge_set_2[id2]:
            c += 1
        elif id3 in dic_edge_set_1[id1] and id3 in dic_edge_set_2[id2]:
            c += 1
    temp.append(c)
    count+=1
    if count%2000==0: print(count/6155.12)

In [119]:
dic_edge_neg_list = {id: [] for id in df['id'].values.tolist()}
dic_edge_neg_set = {id: set([]) for id in df['id'].values.tolist()}
for id1, id2, lab in data_set:
    if lab == 0:
        dic_edge_neg_list[id1].append(id2)
        dic_edge_neg_set[id1].add(id2)
        dic_edge_neg_list[id2].append(id1)
        dic_edge_neg_set[id2].add(id1)

In [100]:
temp = []
for id1, id2 in data:
    c = 0
    for id3 in dic_edge_list[id1]:
        if id3 != id2 and id2 in dic_edge_neg_set[id3]:
            c += 1
    for id3 in dic_edge_list[id2]:
        if id3 != id1 and id1 in dic_edge_neg_set[id3]:
            c += 1
    temp.append(c)

In [104]:
features['edge2_neg'] = temp

0         0
1         0
2         0
3         0
4         0
5         0
6         1
7         0
8         0
9         0
10        0
11        0
12        0
13        0
14        0
15        0
16        0
17        0
18        0
19        0
20        0
21        0
22        0
23        0
24        0
25        0
26        0
27        0
28        0
29        1
         ..
615482    0
615483    0
615484    0
615485    0
615486    0
615487    0
615488    0
615489    0
615490    0
615491    0
615492    0
615493    0
615494    0
615495    0
615496    0
615497    0
615498    0
615499    0
615500    0
615501    0
615502    0
615503    0
615504    0
615505    0
615506    0
615507    0
615508    0
615509    0
615510    0
615511    0
Name: edge2_neg, Length: 615512, dtype: int64

In [123]:
temp = []
count = 0
for id1, id2 in data:
    c = 0
    for id3 in dic_edge_neg_list[id1]:
        if id3 != id2:
            for id4 in dic_edge_list[id3]:
                if id4 != id2 and id4 != id1 and id2 in dic_edge_set[id4]:
                    c += 1
    temp.append(c)
    count += 1
    if count%2000 == 0:
        print(count/6155.12)

0.3249327389230429
0.6498654778460858
0.9747982167691288
1.2997309556921717
1.6246636946152146
1.9495964335382576
2.2745291724613006
2.5994619113843433
2.9243946503073865
3.2493273892304293
3.5742601281534725
3.8991928670765152
4.224125605999558
4.549058344922601
4.873991083845644
5.198923822768687
5.52385656169173
5.848789300614773
6.173722039537815
6.498654778460859
6.823587517383902
7.148520256306945
7.473452995229987
7.7983857341530305
8.123318473076074
8.448251211999116
8.77318395092216
9.098116689845202
9.423049428768245
9.747982167691289
10.072914906614331
10.397847645537373
10.722780384460417
11.04771312338346
11.372645862306502
11.697578601229546
12.022511340152588
12.34744407907563
12.672376817998675
12.997309556921717
13.32224229584476
13.647175034767804
13.972107773690846
14.29704051261389
14.621973251536932
14.946905990459975
15.271838729383019
15.596771468306061
15.921704207229103
16.246636946152147
16.57156968507519
16.896502423998232
17.221435162921274
17.54636790184432

In [124]:
edge3_neg_left = pd.DataFrame()
edge3_neg_left['edge3_neg_left'] = temp
edge3_neg_left.to_pickle('edge3_neg_left.pkl')

In [126]:
temp = []
count = 0
for id1, id2 in data:
    c = 0
    for id3 in dic_edge_list[id1]:
        if id3 != id2:
            for id4 in dic_edge_neg_list[id3]:
                if id4 != id2 and id4 != id1 and id2 in dic_edge_set[id4]:
                    c += 1
    temp.append(c)
    count += 1
    if count%2000 == 0:
        print(count/6155.12)

0.3249327389230429
0.6498654778460858
0.9747982167691288
1.2997309556921717
1.6246636946152146
1.9495964335382576
2.2745291724613006
2.5994619113843433
2.9243946503073865
3.2493273892304293
3.5742601281534725
3.8991928670765152
4.224125605999558
4.549058344922601
4.873991083845644
5.198923822768687
5.52385656169173
5.848789300614773
6.173722039537815
6.498654778460859
6.823587517383902
7.148520256306945
7.473452995229987
7.7983857341530305
8.123318473076074
8.448251211999116
8.77318395092216
9.098116689845202
9.423049428768245
9.747982167691289
10.072914906614331
10.397847645537373
10.722780384460417
11.04771312338346
11.372645862306502
11.697578601229546
12.022511340152588
12.34744407907563
12.672376817998675
12.997309556921717
13.32224229584476
13.647175034767804
13.972107773690846
14.29704051261389
14.621973251536932
14.946905990459975
15.271838729383019
15.596771468306061
15.921704207229103
16.246636946152147
16.57156968507519
16.896502423998232
17.221435162921274
17.54636790184432

In [127]:
edge3_neg_middle = pd.DataFrame()
edge3_neg_middle['edge3_neg_middle'] = temp
edge3_neg_middle.to_pickle('edge3_neg_middle.pkl')

In [129]:
temp = []
count = 0
for id1, id2 in data:
    c = 0
    for id3 in dic_edge_neg_list[id2]:
        if id3 != id1:
            for id4 in dic_edge_list[id3]:
                if id4 != id1 and id4 != id2 and id1 in dic_edge_set[id4]:
                    c += 1
    temp.append(c)
    count += 1
    if count%2000 == 0:
        print(count/6155.12)

0.3249327389230429
0.6498654778460858
0.9747982167691288
1.2997309556921717
1.6246636946152146
1.9495964335382576
2.2745291724613006
2.5994619113843433
2.9243946503073865
3.2493273892304293
3.5742601281534725
3.8991928670765152
4.224125605999558
4.549058344922601
4.873991083845644
5.198923822768687
5.52385656169173
5.848789300614773
6.173722039537815
6.498654778460859
6.823587517383902
7.148520256306945
7.473452995229987
7.7983857341530305
8.123318473076074
8.448251211999116
8.77318395092216
9.098116689845202
9.423049428768245
9.747982167691289
10.072914906614331
10.397847645537373
10.722780384460417
11.04771312338346
11.372645862306502
11.697578601229546
12.022511340152588
12.34744407907563
12.672376817998675
12.997309556921717
13.32224229584476
13.647175034767804
13.972107773690846
14.29704051261389
14.621973251536932
14.946905990459975
15.271838729383019
15.596771468306061
15.921704207229103
16.246636946152147
16.57156968507519
16.896502423998232
17.221435162921274
17.54636790184432

In [135]:
edge3_neg_right = pd.DataFrame()
edge3_neg_right['edge3_neg_right'] = temp
edge3_neg_right.to_pickle('edge3_neg_right.pkl')

In [141]:
features['edge3_neg'] = list(map(lambda x: x[0] + x[1] + x[2], zip(edge3_neg_left['edge3_neg_left'].values.tolist(), edge3_neg_middle['edge3_neg_middle'].values.tolist(), edge3_neg_right['edge3_neg_right'].values.tolist())))
features['edge3_neg_middle'] = edge3_neg_middle['edge3_neg_middle'].values.tolist()
features['edge3_neg_sid'] = list(map(lambda x: x[0] + x[1], zip(edge3_neg_left['edge3_neg_left'].values.tolist(), edge3_neg_right['edge3_neg_right'].values.tolist())))

In [137]:
edge3_neg_right

,edge3_neg_right
0,0
1,3
2,0
3,2
4,0
5,2
6,7
7,0
8,1
9,0


# journal

In [207]:
dic_jrnl = {}
count = 0
for index, row in df.iterrows():
    j = row['journal']
    if isinstance(j, float): continue
    if j not in dic_jrnl:
        dic_jrnl[j] = 1
    else:
        dic_jrnl[j] += 1
    count+=1
print(count)

20298


In [203]:
dic_doc_j = {id: 0 for id in df['id'].values.tolist()}
for id1, id2 in data[:, :]:
    j1 = data_dic[id1][3]
    j2 = data_dic[id2][3]
    if not isinstance(j1, float):
        dic_doc_j[id1] = dic_jrnl[j1]
    if not isinstance(j2, float):
        dic_doc_j[id2] = dic_jrnl[j2]

In [204]:
temp1 = []
temp2 = []
for id1, id2 in data:
    temp1.append(dic_doc_j[id1])
    temp2.append(dic_doc_j[id2])
features['j1'] = temp1
features['j2'] = temp2

In [124]:
features_drop = features.drop(['year_differ','sameAutCount','sameTitleWord','sameT1Ab2','sameT2Ab1','sameAbstract','aut_sum','connection_max',
               'j1', 'j2', 'wmDistance', 'GRU'], axis=1)

In [196]:
temp = []
for id1, id2 in data[:, :]:
    j1 = data_dic[id1][3]
    j2 = data_dic[id2][3]
    if not isinstance(j1, float) and not isinstance(j2, float):
        if j1 == j2:
            temp.append(1)
        else:
            temp.append(-1)
    else:
        temp.append(0)
features['sameJ'] = temp

In [206]:
dic_jrnl

{'"Black': 1,
 '"Electromagnetic': 1,
 '"New': 2,
 '"Noncommutative': 1,
 '"Problems': 1,
 '"Supersymmetries': 1,
 '"Supersymmetry': 1,
 '"Symmetries': 1,
 '"XIIth': 1,
 '"th': 1,
 "'String": 1,
 'AIP': 30,
 'AMS/IP': 1,
 'Acad.': 1,
 'Acta': 54,
 'Adv.': 1,
 'Adv.Appl.Clifford': 3,
 'Adv.Math.': 2,
 'Adv.Sov.Math.': 1,
 'Adv.Theor.Math.Phys.': 126,
 'Alg.Anal.': 3,
 'Alg.Groups': 1,
 'Am.J.Phys.': 4,
 'Am.Math.Soc.Transl.': 2,
 'An.': 1,
 'Anales': 1,
 'Ann.': 14,
 'Ann.Phys.()-': 1,
 'Ann.Rev.Nucl.Part.Sci.': 1,
 'Annalen': 16,
 'Annales': 18,
 'Annals': 207,
 'Aport.': 1,
 'Astron.Astrophys.': 1,
 'Astropart.Phys.': 3,
 'Astrophys.Space': 2,
 'Atti': 1,
 'Austral.J.Phys.': 3,
 'Banach': 2,
 'Boll.Union.Mat.Ital.': 1,
 'Braz.J.Phys.': 12,
 'Brazilian': 1,
 'Bull.': 1,
 'CCAST-WL': 1,
 'CRM': 1,
 'Calogero-Moser': 1,
 'Can.J.Phys.': 5,
 'Cargese': 1,
 'Causality': 1,
 'Central': 1,
 'Chaos': 13,
 'Chin.': 3,
 'Chin.J.Phys.': 4,
 'Chin.Phys.': 1,
 'Chin.Phys.Lett.': 3,
 'Ciencia': 1,
 

** Graph **

In [184]:
G = nx.Graph()
for id in df['id'].values.tolist():
    G.add_node(id)
for id1, id2, lab in data_set:
    if lab==1:
        G.add_edge(id1,id2)

In [52]:
temp = []
count = 0
for id1, id2 in data:
    edge_tag = G.has_edge(id1, id2)
    if edge_tag: G.remove_edge(id1, id2)
    path_tag = nx.has_path(G, id1, id2)
    if path_tag: temp.append(len(shortest_path(G, source=id1, target=id2)))
    else: temp.append(27770)
    if edge_tag: G.add_edge(id1, id2)
    count+=1
    if count%1000==0: print(count/6155.12)

0.16246636946152146
0.3249327389230429
0.4873991083845644
0.6498654778460858
0.8123318473076073
0.9747982167691288
1.1372645862306503
1.2997309556921717
1.4621973251536933
1.6246636946152146
1.7871300640767362
1.9495964335382576
2.112062802999779
2.2745291724613006
2.436995541922822
2.5994619113843433
2.761928280845865
2.9243946503073865
3.0868610197689077
3.2493273892304293
3.411793758691951
3.5742601281534725
3.7367264976149936
3.8991928670765152
4.061659236538037
4.224125605999558
4.38659197546108
4.549058344922601
4.711524714384122
4.873991083845644
5.0364574533071655
5.198923822768687
5.361390192230209
5.52385656169173
5.686322931153251
5.848789300614773
6.011255670076294
6.173722039537815
6.336188408999337
6.498654778460859
6.66112114792238
6.823587517383902
6.986053886845423
7.148520256306945
7.310986625768466
7.473452995229987
7.635919364691509
7.7983857341530305
7.960852103614552
8.123318473076074
8.285784842537595
8.448251211999116
8.610717581460637
8.77318395092216
8.9356503

72.78493351876162
72.94739988822315
73.10986625768466
73.27233262714618
73.4347989966077
73.59726536606922
73.75973173553075
73.92219810499226
74.08466447445379
74.24713084391531
74.40959721337683
74.57206358283835
74.73452995229988
74.89699632176139
75.05946269122292
75.22192906068445
75.38439543014596
75.54686179960748
75.70932816906901
75.87179453853052
76.03426090799205
76.19672727745356
76.35919364691509
76.52166001637661
76.68412638583813
76.84659275529965
77.00905912476118
77.17152549422269
77.33399186368422
77.49645823314574
77.65892460260726
77.82139097206878
77.98385734153031
78.14632371099182
78.30879008045335
78.47125644991488
78.63372281937639
78.79618918883791
78.95865555829943
79.12112192776095
79.28358829722248
79.44605466668399
79.60852103614552
79.77098740560704
79.93345377506856
80.09592014453008
80.25838651399161
80.42085288345312
80.58331925291465
80.74578562237618
80.90825199183769
81.07071836129921
81.23318473076074
81.39565110022225
81.55811746968378
81.72058383

In [53]:
features['path'] = temp

In [66]:
dic_2_connex = {}
count = 0
for id in df['id'].values.tolist():
    sum = 0
    for id2 in G.neighbors(id):
        sum += G.degree(id2)
    dic_2_connex[id] = sum
    count+=1
    if count%1000==0: print(count/6155.12)

0.16246636946152146
0.3249327389230429
0.4873991083845644
0.6498654778460858
0.8123318473076073
0.9747982167691288
1.1372645862306503
1.2997309556921717
1.4621973251536933
1.6246636946152146
1.7871300640767362
1.9495964335382576
2.112062802999779
2.2745291724613006
2.436995541922822
2.5994619113843433
2.761928280845865
2.9243946503073865
3.0868610197689077
3.2493273892304293
3.411793758691951
3.5742601281534725
3.7367264976149936
3.8991928670765152
4.061659236538037
4.224125605999558
4.38659197546108


In [67]:
temp1 = []
temp2 = []
for id1, id2 in data:
    temp1.append(dic_2_connex[id1])
    temp2.append(dic_2_connex[id2])
features['double_connex1'] = temp1
features['double_connex2'] = temp2

In [73]:
temp1 = []
temp2 = []
count = 0
for id1, id2 in data:
    c1, c2 = 0, 0
    for id in dic_edge_list[id1]:
        if id not in dic_edge_set[id2]:
            c1 += 1
    for id in dic_edge_list[id2]:
        if id not in dic_edge_set[id1]:
            c2 += 1
    count+=1
    if count%1000==0: print(count/6155.12)
    temp1.append(c1)
    temp2.append(c2)

0.16246636946152146
0.3249327389230429
0.4873991083845644
0.6498654778460858
0.8123318473076073
0.9747982167691288
1.1372645862306503
1.2997309556921717
1.4621973251536933
1.6246636946152146
1.7871300640767362
1.9495964335382576
2.112062802999779
2.2745291724613006
2.436995541922822
2.5994619113843433
2.761928280845865
2.9243946503073865
3.0868610197689077
3.2493273892304293
3.411793758691951
3.5742601281534725
3.7367264976149936
3.8991928670765152
4.061659236538037
4.224125605999558
4.38659197546108
4.549058344922601
4.711524714384122
4.873991083845644
5.0364574533071655
5.198923822768687
5.361390192230209
5.52385656169173
5.686322931153251
5.848789300614773
6.011255670076294
6.173722039537815
6.336188408999337
6.498654778460859
6.66112114792238
6.823587517383902
6.986053886845423
7.148520256306945
7.310986625768466
7.473452995229987
7.635919364691509
7.7983857341530305
7.960852103614552
8.123318473076074
8.285784842537595
8.448251211999116
8.610717581460637
8.77318395092216
8.9356503

73.10986625768466
73.27233262714618
73.4347989966077
73.59726536606922
73.75973173553075
73.92219810499226
74.08466447445379
74.24713084391531
74.40959721337683
74.57206358283835
74.73452995229988
74.89699632176139
75.05946269122292
75.22192906068445
75.38439543014596
75.54686179960748
75.70932816906901
75.87179453853052
76.03426090799205
76.19672727745356
76.35919364691509
76.52166001637661
76.68412638583813
76.84659275529965
77.00905912476118
77.17152549422269
77.33399186368422
77.49645823314574
77.65892460260726
77.82139097206878
77.98385734153031
78.14632371099182
78.30879008045335
78.47125644991488
78.63372281937639
78.79618918883791
78.95865555829943
79.12112192776095
79.28358829722248
79.44605466668399
79.60852103614552
79.77098740560704
79.93345377506856
80.09592014453008
80.25838651399161
80.42085288345312
80.58331925291465
80.74578562237618
80.90825199183769
81.07071836129921
81.23318473076074
81.39565110022225
81.55811746968378
81.72058383914529
81.88305020860682
82.04551657

In [74]:
features['nocom_connex1'] = temp1
features['nocom_connex2'] = temp2

In [72]:
temp = []
count = 0
for id1, id2 in data:
    c = 0
    for id3 in G.neighbors(id1):
        if id3 != id2:
            for id4 in G.neighbors(id3):
                if id4 != id1 and id4 != id2 and id4 in G.neighbors(id2):
                        c += 1
    temp.append(c)
    count+=1
    if count%1000==0: print(count/6155.12)

KeyboardInterrupt: 

# Text mining

In [47]:
temp = []
for id1, id2 in data:
    title1 = data_dic[id1][1].split()
    title2 = data_dic[id2][1].split()
    count = 0
    for a1 in title1:
        for a2 in title2:
            if a1 == a2 and a1 not in stop:
                count += 1
    temp.append(count)
features['T1T2_c'] = temp

In [48]:
temp = []
for id1, id2 in data:
    title1 = nltk.word_tokenize(data_dic[id1][1])
    title2 = nltk.word_tokenize(data_dic[id2][4])
    count = 0
    for a1 in title1:
        for a2 in title2:
            if a1 == a2 and a1 not in stop:
                count += 1
    temp.append(count)
features['T1Ab2_c'] = temp

KeyboardInterrupt: 

In [32]:
temp = []
for id1, id2 in data:
    title1 = nltk.word_tokenize(data_dic[id1][4])
    title2 = nltk.word_tokenize(data_dic[id2][1])
    count = 0
    for a1 in title1:
        for a2 in title2:
            if a1 == a2 and a1 not in stop:
                count += 1
    temp.append(count)
features['T2Ab1_c'] = temp

In [35]:
temp = []
for id1, id2 in data:
    title1 = nltk.word_tokenize(data_dic[id1][4])
    title2 = nltk.word_tokenize(data_dic[id2][4])
    count = 0
    for a1 in title1:
        for a2 in title2:
            if a1 == a2 and a1 not in stop:
                count += 1
    temp.append(count)
features['Ab1Ab2_c'] = temp

In [83]:
features = features.drop(['edge_combine'], axis=1)

In [24]:
features.to_pickle('features.pkl')

In [25]:
features[['edge2','edge2_dir']]

,edge2,edge2_dir
0,1,0
1,20,6
2,0,0
3,0,0
4,0,0
5,14,0
6,12,7
7,0,0
8,5,1
9,0,0


# NLP preprocessing

In [10]:
corpus = [df["title"].values.tolist()[i]+ ' ' +df["abstract"].values.tolist()[i] for i in range(df.shape[0])]

In [11]:
tok_corp = [nltk.word_tokenize(sent) for sent in corpus]

# Word2Vec

In [141]:
model_word = gensim.models.Word2Vec(tok_corp, min_count=1, size = 100)
#model.wv.wmdistance(df["title"].values[0],df["title"].values[1])

In [142]:
features['wmDistance'] = [model_word.wv.wmdistance(data_dic[id1][1], data_dic[id2][1]) for id1, id2 in data]

In [ ]:
features['AbstractWmDistance'] = [model_word.wv.wmdistance(data_dic[id1][4], data_dic[id2][4]) for id1, id2 in data]

In [151]:
model_word.init_sims(replace=True)
for i in range(50):
    print(i+1,model_word.wv.wmdistance(data_dic[data[i,0]][4], data_dic[data[i,1]][4]))

1 0.1339298271328601
2 0.14167229115312266
3 0.12307047666299836
4 0.1103670142742237
5 0.09899311415005364
6 0.10080362438932107
7 0.0910934697826492
8 0.13462998467393508
9 0.11522108130367496
10 0.14076962499877868
11 0.15752950663721732
12 0.127035484351778
13 0.08497872486363087
14 0.10961360493449714
15 0.11416227830500923
16 0.07358501520007174
17 0.1312186151828572
18 0.16146123620169012
19 0.12882718779283137
20 0.14423563405745668
21 0.13278500492518666
22 0.13480869669106998
23 0.09217015082733772
24 0.1608324591017828
25 0.10022493058585812
26 0.14064141021434784
27 0.13123659533039306
28 0.16731092814682172
29 0.11317491446457001
30 0.14592952888476
31 0.12551404960977883
32 0.13539734224223243
33 0.18888623662269347
34 0.11376379370554371
35 0.14330909607894304
36 0.11735796579703131
37 0.09876888465577194
38 0.125333636854385
39 0.11345312104258164
40 0.14901826464526483
41 0.15832069077165645
42 0.13550054657877006
43 0.13035887098890125
44 0.11988842270139112
45 0.0777

# Doc2Vec

In [12]:
train_corpus = [TaggedDocument(doc, [i]) for i, doc in enumerate(tok_corp)]
model_doc = Doc2Vec(vector_size=100, min_count=1, epochs=50)
model_doc.build_vocab(train_corpus)

In [13]:
%time model_doc.train(train_corpus, total_examples=model_doc.corpus_count, epochs=model_doc.epochs)
model_doc.save('model_doc')

Wall time: 2min 1s


In [29]:
model_doc.infer_vector(tok_corp[1]).tolist()

[1.474726915359497,
 0.34120360016822815,
 1.0735846757888794,
 1.0724287033081055,
 -0.8355870246887207,
 1.195784330368042,
 -0.5436702966690063,
 -2.3248977661132812,
 2.0581917762756348,
 0.6549327969551086,
 -1.5657272338867188,
 1.1949348449707031,
 -0.6747865080833435,
 1.1150985956192017,
 0.41826173663139343,
 -1.119102120399475,
 0.1606740802526474,
 0.3114176392555237,
 -0.2151210904121399,
 0.7164869904518127,
 0.40876078605651855,
 -2.239044427871704,
 1.110196828842163,
 -1.0339666604995728,
 -0.3278810679912567,
 1.3012611865997314,
 1.3370336294174194,
 -1.939644694328308,
 -1.4356322288513184,
 -1.5471018552780151,
 -0.12631256878376007,
 -0.5129476189613342,
 0.06311173737049103,
 2.229620933532715,
 0.08299599587917328,
 0.40562206506729126,
 -0.6226986646652222,
 0.5628371238708496,
 0.7238472104072571,
 -1.409699559211731,
 0.6891328692436218,
 -1.4387511014938354,
 1.427460789680481,
 -0.9054232239723206,
 -0.3539733290672302,
 1.0911692380905151,
 -1.085486769676

In [30]:
doc_vec_arr = np.array([model_doc.infer_vector(tok_corp[i]).tolist() for i in range(df.shape[0])])

In [33]:
svd = TruncatedSVD(n_components=50, n_iter=10, random_state=42)
doc_vec_svd = svd.fit_transform(doc_vec_arr)

In [34]:
list_id = df['id'].values.tolist()
dic_d2v = {list_id[i]: doc_vec_svd[i] for i in range(df.shape[0])}

In [36]:
temp1, temp2 = [], []
for id1, id2 in data:
    temp1.append(dic_d2v[id1].tolist())
    temp2.append(dic_d2v[id2].tolist())

In [37]:
d2v_features1 = pd.DataFrame(np.array(temp1))
d2v_features2 = pd.DataFrame(np.array(temp2))
d2v_features1.to_pickle('d2v1.pkl')
d2v_features2.to_pickle('d2v2.pkl')

In [41]:
temp1, temp2 = [], []
for id1, id2 in test:
    temp1.append(dic_d2v[id1].tolist())
    temp2.append(dic_d2v[id2].tolist())

In [42]:
d2v_features_test1 = pd.DataFrame(np.array(temp1))
d2v_features_test2 = pd.DataFrame(np.array(temp2))
d2v_features_test1.to_pickle('d2v_test1.pkl')
d2v_features_test2.to_pickle('d2v_test2.pkl')

In [43]:
d2v_features_test2.shape

(32648, 50)

In [152]:
temp = pd.read_csv('output_train.csv')

In [154]:
features['GRU'] = temp['category']

In [125]:
features_standard = pd.DataFrame(sklearn.preprocessing.scale(features_drop.values), columns = features_drop.columns.tolist())

D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [128]:
features_standard.to_pickle('f_standard.pkl')

In [46]:
features

,year1,year2,year_differ,sameAutCount,sameTitleWord,sameT1Ab2,sameT2Ab1,sameAbstract,wmDistance,GRU,...,edge2_sqr_weighted,connex_max,connex_min,year_max,year_min,->D->,->D<-,<-D->,src1,tgt2
0,1995,1995,0,-1,2,3,4,6,3.867921,1,...,49,12,6,1995,1995,0,1,0,3,8
1,1997,1996,1,0,0,1,0,6,7.302732,1,...,770973,147,79,1997,1996,6,8,6,68,124
2,1993,1995,2,0,0,3,0,7,4.217448,0,...,0,5,1,1995,1993,0,0,0,0,2
3,1999,2003,4,0,0,1,1,7,5.294790,0,...,0,20,14,2003,1999,0,0,0,16,2
4,1997,2002,5,-1,0,0,1,7,3.839918,0,...,0,24,7,2002,1997,0,0,0,0,2
5,1997,1997,0,-1,0,10,4,41,5.308518,1,...,591194,38,35,1997,1997,0,7,7,29,16
6,1999,1995,4,-1,0,0,2,8,4.923457,1,...,2059887,739,61,1999,1995,7,1,4,25,726
7,2002,1995,7,-1,1,3,3,12,3.709659,0,...,0,86,15,2002,1995,0,0,0,15,63
8,1997,1997,0,0,0,0,0,1,3.081206,1,...,1039595,100,41,1997,1997,1,4,0,33,76
9,2000,1992,8,0,0,1,0,4,4.183478,0,...,0,30,4,2000,1992,0,0,0,1,30


In [40]:
features[['edge2','->D->','->D<-','<-D->']]

,edge2,->D->,->D<-,<-D->
0,1,0,1,0
1,20,6,8,6
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0
5,14,0,7,7
6,12,7,1,4
7,0,0,0,0
8,5,1,4,0
9,0,0,0,0
